In [ ]:
# !python -m pip install yahooquery --q

In [6]:
import os
import mlflow
from mlflow.langchain import autolog

from yahooquery import Ticker
import requests

from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

In [7]:
MODEL_NAME = 'gemini-2.0-flash'
# COMPANY_NAME = 'Apple Inc.'

In [8]:
mlflow.set_tracking_uri("http://20.75.92.162:5000/")

### 1. Create a Prompt Template

In [23]:
initial_template = """You are a financial analyst specializing in market sentiment analysis. 
Analyze the following news data about {company_name} (stock code: {stock_code} and provide a structured sentiment analysis.

News Data:{news_data}

Please provide your analysis in the following JSON format ONLY:
{{
    "company_name": The above company name,
    "stock_code": The above stock_code,
    "newsdesc": "Brief summary of the news (max 200 words)",
    "sentiment": "Positive/Negative/Neutral",
    "people_names": ["list of people mentioned in the news"],
    "places_names": ["list of places/locations mentioned"],
    "other_companies_referred": ["list of other companies mentioned"],
    "related_industries": ["list of related industries mentioned"],
    "market_implications": "Analysis of potential market impact (max 150 words)",
    "confidence_score": 0.0
}}

Guidelines:
- confidence_score should be between 0.0 and 1.0 (1.0 = very confident)
- sentiment should be based on overall tone and potential market impact
- Extract only relevant named entities
- Focus on factual analysis over speculation
- If news data is limited, indicate this in your analysis

Respond with ONLY the JSON object, no additional text."""

In [17]:
# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name="rvg-market-sentiment-analyzer",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="updated",
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "author": "RVG",
        "task": "Market Sentiment Analysis",
        "language": "en",
        'llm': 'gemini-2.0-flash'
    },
)

2025/09/21 17:27:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: rvg-market-sentiment-analyzer, version 8


In [9]:

company_to_ticker = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Amazon": "AMZN",
    "Alphabet (Google)": "GOOGL",
    "Tesla": "TSLA",
    "Meta (Facebook)": "META",
    "Berkshire Hathaway": "BRK.B",
    "NVIDIA": "NVDA",
    "Johnson & Johnson": "JNJ",
    "JPMorgan Chase": "JPM",
    "Visa": "V",
    "Procter & Gamble": "PG",
    "UnitedHealth Group": "UNH",
    "Home Depot": "HD",
    "Mastercard": "MA",
    "Bank of America": "BAC",
    "Pfizer": "PFE",
    "Walt Disney": "DIS",
    "Chevron": "CVX",
    "Coca-Cola": "KO",
    "Verizon": "VZ",
    "Cisco Systems": "CSCO",
    "Comcast": "CMCSA",
    "PepsiCo": "PEP",
    "AT&T": "T",
    "Intel": "INTC",
    "Netflix": "NFLX",
    "Adobe": "ADBE",
    "Salesforce": "CRM",
    "Abbott Laboratories": "ABT",
    "Nike": "NKE",
    "Eli Lilly": "LLY",
    "Wells Fargo": "WFC",
    "Oracle": "ORCL",
    "McDonald's": "MCD",
    "IBM": "IBM",
    "Merck": "MRK",
    "PayPal": "PYPL",
    "Costco": "COST",
    "Broadcom": "AVGO",
    "Starbucks": "SBUX",
    "General Electric": "GE",
    "Goldman Sachs": "GS",
    "Boeing": "BA",
    "American Express": "AXP",
    "Morgan Stanley": "MS",
    "Walmart": "WMT",
    "3M": "MMM",
    "Caterpillar": "CAT",
    "General Motors": "GM"
}

def get_stock_code_from_company_name(company_name:str) -> str:
    return company_to_ticker[company_name]

In [10]:

def get_headline_news(company_name: str) -> str:
    stock_code = get_stock_code_from_company_name(company_name)
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock_code}&apikey=IPXL2Z8EW349Q0B9"
    response = requests.get(url)
    data = response.json()
    print(data)
    headlines = []
    for article in data['feed'][:5]:
        headlines.append(article['summary'])
    return '\n '.join(headlines),stock_code


In [ ]:

company_name = 'IBM'
news_data,stock_code = get_headline_news(company_name)
# print(news_data)
# print()
# print(stock_code)

my_prompt = mlflow.genai.load_prompt("prompts:/rvg-market-sentiment-analyzer/8")
system_prompt = my_prompt.template

analyzer_prompt = ChatPromptTemplate.from_template(system_prompt)

model = init_chat_model(MODEL_NAME, model_provider="google_genai")

parser = StrOutputParser()


chain =  analyzer_prompt | model | parser 

# print(news_data)

result = chain.invoke({"company_name":company_name,
                       "stock_code":stock_code,
                       "news_data":news_data})

print(result)

```json
{
    "company_name": "IBM",
    "stock_code": "IBM",
    "newsdesc": "IBM is pursuing an aggressive strategy in AI and quantum computing via partnerships. They are expanding their AI presence in India. IBM's annual Cost of a Data Breach Report indicates AI adoption outpaces security controls. A strong second-quarter earnings report showed revenue growth in Software and Infrastructure.",
    "sentiment": "Positive",
    "people_names": [],
    "places_names": ["India"],
    "other_companies_referred": [],
    "related_industries": ["AI", "quantum computing", "Software", "Infrastructure", "Data Breach", "Security"],
    "market_implications": "The news suggests a positive outlook for IBM. Aggressive expansion in AI and quantum computing, coupled with strong earnings, signals growth potential. However, the Data Breach Report highlights a risk associated with rapid AI adoption and potential security vulnerabilities, which could temper investor enthusiasm to a small degree.",
    "